In [1]:
import numpy as np
import os
import torch
import yaml
import json
import random 
import pandas as pd

from sklearn.externals import joblib
from sklearn.model_selection import ParameterGrid

In [2]:
random.seed(234)
np.random.seed(5432)

In [3]:
data_path = './data/'
label_path = os.path.join(data_path, 'labels')

In [4]:
def yaml_write(x, path):
    with open(path, 'w') as fp:
        yaml.dump(x, fp)
        
def yaml_read(path):
    with open(path, 'r') as fp:
        return yaml.load(fp)

In [5]:
param_grid = {
    'lr_final_classifier' : [1e-2, 1e-3, 1e-4],
    'lambda_final_classifier_cf' : [1e1, 1e0, 1e-1, 0.0],
    'lambda_clp' : [1e1, 1e0, 1e-1, 1e-2, 0.0],
    'lambda_clp_entropy' : [0.0],
    'num_epochs' : [20],
    'weighted' : [False],
    'num_samples_eval': [1],
    'cf_gradients' : [True, False]
}

the_grid = list(ParameterGrid(param_grid))
np.random.shuffle(the_grid)

In [6]:
for sensitive_variable in ['age', 'race_eth', 'gender']:
    for outcome in ['los', 'mortality']:
        config_load_path = os.path.join(data_path, 'config', 'defaults', 'cfvae', outcome, sensitive_variable, 'model_config.yaml')

        config_path = os.path.join(data_path, 'config', 'grid', 'final_classifier', outcome, sensitive_variable)
        os.makedirs(config_path, exist_ok = True)

        grid_df = pd.DataFrame(the_grid)
        grid_df.to_csv(os.path.join(config_path, 'config.csv'), index_label = 'id')

        for i, config_dict in enumerate(the_grid):
            config_dict_vae = yaml_read(config_load_path)
            config_dict_vae.update(config_dict)
            yaml_write(config_dict_vae, os.path.join(config_path, '{}.yaml'.format(i)))